# **Analyse: Räumliche Verteilung der Periodika in digiPress**

## Installation benötigter Bibliotheken

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt # Zum Erstellen von Diagrammen
import numpy as np # Bibliothek für diverse Datenstrukturen und numerische Berechnungen
import re # Bibliothek für Regular Expressions (RegEx), auch reguläre Ausdrücke genannt
import time # Zum Einbauen von Pausen bei HTTP-Anfragen
from functools import lru_cache # Bibliothek für Cache
import ast # Zur Datentransformation

## Daten einlesen

In [ ]:
df = pd.read_excel('Zeitungsliste.xlsx')
df.head()

### Anfangs- und Endzeitpunkt des Erscheinungszeitraums trennen

In [ ]:
# Gleiche Vorgehensweise wie bei Orten: hier Trennung bei Bindestrich

df[["Start", "End"]] = df["Erscheinungsverlauf"].str.split("–", expand=True)
df["Start"] = df["Start"].str.strip().astype(int) # .astype(int) = Umwandlung in Integers (int)
df["End"] = df["End"].str.strip().astype(int)

In [ ]:
# Überprüfung: Folgen alle Zeilen der Spalte "Erscheinungsverlauf" dem Muster "JJJJ - JJJJ"?

regex_pattern = r'^\d{4}\s*–\s*\d{4}$' # Regulärer Ausdruck für "JJJJ - JJJJ"
# ^ markiert Anfang
# \d = Zahl (digit)
# {4} spezifiziert Anzahl
# \s* erlaubt optionale Leerzeichen
# $ markiert Ende

# Anwendung des Patterns auf alle Werte: trifft Muster zu?
pattern = df['Erscheinungsverlauf'].astype(str).apply(lambda x: bool(re.match(regex_pattern, x)))

if pattern.all(): # Pattern trifft auf alle Zeilen zu
    print("Alle Werte in 'Erscheinungsverlauf' folgen dem Muster 'JJJJ – JJJJ'.")
else: #Pattern trifft auf einzelne Zeilen NICHT zu
    print("Diese Werte folgen NICHT dem Muster 'JJJJ – JJJJ':")
    non_matching_values = df.loc[~pattern, 'Erscheinungsverlauf'] # Filtere das Datenset nach Zeilen, wo das Muster NICHT (~) befolgt wird
    for val in non_matching_values: # Gib diese Werte aus
        print(val)

In [ ]:
# Korrigierter Code

df[["Start", "End"]] = df["Erscheinungsverlauf"].str.split("–", expand=True)
df["Start"] = df["Start"].str.strip().astype(int)
df["End"] = df["End"].str.strip().replace("", "2025").astype(int) # Leere Enddatumsangaben werden durch "2025" ersetzt

## Wann sind die in digiPress verfügbaren Periodika erschienen?

Auswertung auf Basis des angegebenen Erscheinungsverlaufs (!): Wo gibt es zeitliche Häufungen, d.h. in welchen Zeiträumen sind besonders viele der digitalisierten Zeitungen erschienen?

### Jahre und Anzahl erschienener Titel pro Jahr eruieren

In [ ]:
#Ordnen der Daten nach Startjahr
sorted_data = df.sort_values(by='Start').reset_index(drop=True)

# Erstellen von Liste an durchzugehenden Jahre
years = np.arange(sorted_data['Start'].min(), sorted_data['End'].max() + 1)

# Anzahl aktiver Zeitungsunternehmen pro Jahr berechnen
active_counts = []
for y in years: #
    # alle Titel, die in diesem Jahr aktiv sind, finden:
    active_titles = sorted_data[(sorted_data['Start'] <= y) & (sorted_data['End'] >= y)]
    # hierunter jedes Unternehmen nur einmal zählen (z.B. wenn Beilage und Hauptblatt eines Unternehmens im gleichen Jahr erschienen sind):
    n_active_companies = active_titles['digiPress-ID'].nunique()
    # zu Gesamtcounts hinzufügen
    active_counts.append(n_active_companies)

# Beispielausschnitte aus erstellten Überblicken
print(years[51:60])
print(active_counts[51:60])

### Darstellung als Flächendiagramm

In [ ]:
plt.figure(figsize=(14,6)) # Initalisieren des Diagramms und Größeneinstellung
plt.plot(years, active_counts, color="green") # Hinzufügen der Daten = Linie und Achsen (aktive Titel pro Jahr)
plt.fill_between(years, active_counts, alpha=0.3, color="green") # Füllung zwischen x-Achse und Linie, alpha setzt Transparenz
plt.title("Erscheinungsverlauf der Periodika in digiPress: Unternehmen pro Jahr", fontsize=20) # Diagrammtitel
plt.xlabel("Jahr") # Label für x-Achse
plt.ylabel("Anzahl erschienener Titel", fontsize=16) # Label für y-Achse
plt.grid(True, alpha=0.5, linestyle="--") # Angaben für Hintergrundraster
plt.show()

### Darstellung als Heatmap

In [ ]:
heatmap = np.array([active_counts]) # Umwandlung in korrektes Format für Heatmap
plt.figure(figsize=(16, 3)) # Größe des Diagramms
plt.imshow(heatmap, aspect="auto", cmap="Greens",
           extent=[years.min(), years.max(), 0, 1]) # Erstellen von Diagramm inklusive Festlegen der Gestaltung
plt.colorbar(label="Anzahl erschienener Titel") # Hinzufügen von Legende mit Label
plt.title("Heatmap des Erscheinungsverlauf der Periodika in digiPress: Unternehmen pro Jahr", fontsize=18) # Überschrift
plt.xlabel("Jahr") # Label für x-Achse
plt.yticks([])  # Entfernen von y-Achse (nicht relevant hier)
plt.show() # Diagramm anzeigen

**Frage:** Was zeigen uns diese Überblicksgrafiken? Was sagen sie über die Daten in digiPress aus, was nicht?

## Für welche Jahrgänge sind in digiPress Ausgaben verfügbar?

**Frage:** Wie könnten wir - ausgehend von der gescrapten Zeitungsliste - erheben, für welche Jahre der verschiedenen Zeitungsunternehmen digitale Ausgaben zur Verfügung stehen? (Tipp: Denkt an die verschiedenen digiPress-Einstiegsmöglichkeiten.)

### Erheben der verfügbaren Jahrgänge

In [ ]:
# Definieren einer Funktion, die eine URL als Input nimmt und das HTML zurückgibt
# Rückschau: wie bei anfänglichem Webscraper in Notebook 1

def get_html_from_url(url):
    try:
      response = requests.get(url)
      response.raise_for_status()  # Fehlermeldungen rückmelden

      soup = BeautifulSoup(response.content, "html.parser")
      return soup  # BeautifulSoup-Objekt ausgeben

    except requests.exceptions.RequestException as e: # Falls ein Fehler auftritt:
      print(f"Error fetching URL {url}: {e}") # Fehler ausgeben
      return None # leeren Output zurückgeben

**Aufgabe:** Untersucht das HTML der folgenden Website: [https://digipress.digitale-sammlungen.de/calendar/newspaper/bsbmult00000618](https://digipress.digitale-sammlungen.de/calendar/newspaper/bsbmult00000618) - was unterscheidet Jahre mit verfügbaren digitalen Ausgaben von Jahren ohne Ausgaben?

In [ ]:
# Webscraper für Kalenderansicht

@lru_cache(maxsize=None) # Cache (merkt sich Ergebnisse für bereits abgefragte URLs) - dieses Mal über eine bestehende Bibliothek (vgl. Notebook 2a)

def get_available_years(url):
  print(f"Wird abgerufen: {url}") # zur Übersicht über Fortschritt

  time.sleep(0.5) # Pause einbauen (hier 0.5 Sekunden) - verhindert zu schnelle Anfragen

  soup = get_html_from_url(url) # HTML der URL laden - über vorher definierte Funktion

  # Finde alle <ul>-Elemente mit der Klasse "centuryYearList clearfix"
  ul_elements = soup.find_all('ul', class_='centuryYearList clearfix')

  # Leere Liste zum Speichern der Links
  available_years = []

  for ul_element in ul_elements: # Für jedes gefundene <ul>-Element:
      list_items = ul_element.find_all('li') # Finde alle sich <li>-Elemente darin
      for item in list_items: # Für jedes gefundene <li>-Element:
          link = item.find('a') # Finde den Link (<a>-Element)
          if link and 'href' in link.attrs: # Falls Link vorhanden
              year = link.text.strip() # Jahresangabe = Text von Link
              available_years.append((year)) # Zu verfügbaren Jahren hinzufügen
  return available_years # Liste verfügbarer Jahre zurückgeben

In [ ]:
# Funktion pro Zeile durchführen - je mit "Link" (= URL der Kalenderansicht) als Input - läuft ca. 7-10 Minuten
df['available_years'] = df['Link'].apply(get_available_years)

In [ ]:
# Alternative (falls Webscraping zu lange lädt): Vorbereitete Version der Zeitungsliste laden

df = pd.read_excel("Zeitungsliste_verfügbare_Jahrgänge.xlsx")

In [ ]:
# Zwischeneinblick in Daten: Was hat sich geändert?
df.head()

### Vergleich von erschienen versus in digiPress verfügbaren Jahrgängen

**Erinnerung:** "Verfügbar" meint hier mindestens eine Ausgabe dieses Jahrgangs ist digital zugänglich. Als Perspektive: Genaue Ausgabenzahlen wären ein möglichster nächster Schritt und würden ein erneues Webscraping pro Periodikum und Jahr bedeuten.

In [ ]:
# Umwandeln der Strings (z.B. "1817") in "available_years" in Zahlen
df['available_years'] = df['available_years'].apply(
    lambda lst: [int(y) for y in ast.literal_eval(lst)]
)

# Für jedes Jahr überprüfen: Bei wievielen unterschiedlichen Zeitungsunternehmen ist das Jahr in der Liste "available_year"?
active_counts_available = []
for y in years:
    active_titles = df[df['available_years'].apply(lambda lst: y in lst)]
    n_active_companies = active_titles['digiPress-ID'].nunique()
    active_counts_available.append(n_active_companies)

# Diagrammerstellung

plt.figure(figsize=(14,6)) # Initialisierung und Diagrammgröße

# Daten von vorherigem Flächendiagramm
plt.plot(years, active_counts, color="green", label="Erscheinungsverlauf")
plt.fill_between(years, active_counts, alpha=0.3, color="green")

# neue Daten von tatsächlich verfügbaren Jahrgängen hinzufügen
plt.plot(years, active_counts_available, color="blue", label="Verfügbare Jahrgänge")
plt.fill_between(years, active_counts_available, alpha=0.3, color="blue")

# Überschrift, Labels, Legende etc. hinzufügen
plt.title("Erscheinungsverlauf versus Verfügbarkeit der Periodika in digiPress: Unternehmen pro Jahr", fontsize=20)
plt.xlabel("Jahr")
plt.ylabel("Anzahl erschienener Unternehmen", fontsize=16)
plt.grid(True, alpha=0.5, linestyle="--")
plt.legend()
plt.show()

## Speicherung als Excel-Dateien

In [ ]:
df.to_excel("Zeitungsliste_verfügbare_Jahrgänge.xlsx", index=False)

## Perspektiven

**Fragen:**
* Worüber gibt uns die obige Grafik Auskunft?
* Welche weiteren Auswertungen könnten/sollten hieran und an die explorativen Analysen der letzten Notebooks anschließen?
* Welche Chancen bietet die Analyse von gescrapten Metadaten?
* Welche Limitationen hat das Scrapen von Zeitungsportalen?